# VQE on Qnexus using native gates
This notebook aims to run VQE on quantinuum to provide a warm start for DBQA.

## Contents

1. [VQE ansatz circuit](#VQE_ansatz)<br>

# 1. VQE ansatz circuit
First, we look at an example circuit taken from qibo's AVVQE example (hdw_efficient)
and for better compilation results, we replace the $CX$ gates with the native $RZZ$.

In [37]:
from pytket import Circuit, OpType
from pytket.circuit.display import render_circuit_jupyter
import numpy as np
import scipy
from sympy import Symbol
import matplotlib.pyplot as plt


## 1.1 2-qubit VQE circuit

We start with the simple case where there are only 2 qubits.

In [49]:
symbols = [Symbol(f"p{i}") for i in range(5)]
symbolic_circuit = Circuit(2)
symbolic_circuit.X(0)
symbolic_circuit.Ry(symbols[0], 0).Ry(symbols[1], 1)
symbolic_circuit.ZZPhase(symbols[4],0, 1)
symbolic_circuit.Ry(symbols[2], 0).Ry(symbols[3], 0)

[X q[0]; Ry(p1) q[1]; Ry(p0) q[0]; ZZPhase(p4) q[0], q[1]; Ry(p2) q[0]; Ry(p3) q[0]; ]

In [56]:
symbolic_circuit.free_symbols()

{p0, p1, p2, p3, p4}

In [51]:
# render_circuit_jupyter(symbolic_circuit)

In [54]:
# Hamiltonian definition
from pytket.pauli import Pauli, QubitPauliString
from pytket.circuit import Qubit
from pytket.utils.operators import QubitPauliOperator
# XXZ model
delta = 0.5
term_sum = {}
term_sum.update({QubitPauliString({Qubit(0): Pauli.X, Qubit(1): Pauli.X}): 1})
term_sum.update({QubitPauliString({Qubit(0): Pauli.Y, Qubit(1): Pauli.Y}): 1})
term_sum.update({QubitPauliString({Qubit(0): Pauli.Z, Qubit(1): Pauli.Z}): delta})
H = QubitPauliOperator(term_sum)


### 1.1.1 Numerical optimisation

In [59]:
def substitute_ansatz_circuit(symbolic_circuit, params):
    assert len(params) == len(symbolic_circuit.free_symbols())
    symbol_dict = {s: p for s, p in zip(symbolic_circuit.free_symbols(), params)}
    state_prep_circuit = symbolic_circuit.copy()
    state_prep_circuit.symbol_substitution(symbol_dict)
    return state_prep_circuit

In [63]:
def unitary_expectation(H, U=None, ket0=None):
    if ket0 is None:
        ket0 = np.zeros((H.shape[0],), dtype=complex)
        ket0[0] = 1.0
    if U is None:
        psi = ket0
    else:
        psi = U @ ket0
    return np.vdot(psi, H @ psi).real

In [66]:
from copy import copy
from qibo import hamiltonians
from qibo.symbols import X, Y, Z
# build zero state
zero_state = np.zeros(4)
# initial params
params_len = len(symbols)
# fix numpy seed to ensure replicability of the experiment
seed = 10
np.random.seed(seed)
initial_params = np.random.uniform(-np.pi, np.pi, params_len)
print('Initial parameters:', initial_params)
# h hamiltonian
h_qibo = hamiltonians.SymbolicHamiltonian(X(0)*X(1) + Y(0)*Y(1) + delta * Z(0)*Z(1))
target_energy = np.real(np.min(np.asarray(h_qibo.eigenvalues())))
# initial energy
c = substitute_ansatz_circuit(symbolic_circuit, initial_params)
initial_energy = unitary_expectation(h_qibo.matrix, c.get_unitary())
print('Target enegry:', target_energy)
print('Initial energy:', initial_energy)
print('Net difference:', initial_energy-target_energy)

[Qibo 0.2.13|WARNING|2025-01-28 12:14:57]: Calculating the dense form of a symbolic Hamiltonian. This operation is memory inefficient.


Initial parameters: [ 1.70475788 -3.01120431  0.83973663  1.5632809  -0.00938072]
Target enegry: -2.5
Initial energy: 0.047693865228487134
Net difference: 2.5476938652284873


## 1.2 n-qubit VQE circuit